
## GPT from scratch in PyTorch


In [1]:
#!pip install torch
import torch
import numpy as np
import torch.nn as nn

from torch.nn import functional as F


In [2]:
#pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

In [3]:

torch.manual_seed(256)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size        = 60#40      ## N tokens in sequence
batch_size        = 64 
max_iters         = 7000#6000
eval_interval     = 500     
learning_rate     = 0.0001
eval_iters        = 300
vocab_size        = 65

## every id for a given token is embedded to vector of this size
n_embd            = 768 #728                 
n_head            = 12 #8         ## 8 attention heads
n_layer           = 8 #6         ## 6 eoncoder layers
dropout           = 0.15 #0.2


In [4]:

text = ''

input_file2 = 'Final_dataset.txt' 

with open(input_file2, 'r', encoding='utf-8') as f:
    text = f.read()


In [5]:

print("length of data in letter or characters")
len(text)




length of data in letter or characters


10429116

In [6]:

list(set(text))


['&',
 '’',
 '\xa0',
 'p',
 'm',
 'F',
 '*',
 'u',
 'x',
 '•',
 '8',
 'ï',
 '@',
 'º',
 'M',
 'ē',
 'Ò',
 'h',
 '5',
 '7',
 '+',
 "'",
 ' ',
 '®',
 'ﬁ',
 'C',
 '“',
 'i',
 '9',
 'W',
 '3',
 'é',
 '×',
 ',',
 '>',
 '\n',
 '4',
 'X',
 'ō',
 '½',
 'U',
 '°',
 'K',
 'B',
 'P',
 '[',
 '”',
 'V',
 ';',
 '(',
 'b',
 'E',
 '™',
 'y',
 'ĕ',
 'Z',
 '¾',
 '⁰',
 '/',
 'q',
 '6',
 'O',
 ']',
 '#',
 ')',
 '‘',
 'r',
 'o',
 'f',
 'g',
 '$',
 '=',
 'L',
 'J',
 '_',
 '‒',
 '―',
 ':',
 '.',
 'k',
 '-',
 'N',
 'μ',
 's',
 'n',
 'R',
 'a',
 'l',
 '2',
 '…',
 '<',
 'c',
 'w',
 'G',
 '—',
 'T',
 'j',
 '–',
 'e',
 't',
 '!',
 '¼',
 '1',
 '?',
 'I',
 'A',
 'Q',
 'H',
 'z',
 '%',
 '©',
 'Y',
 'd',
 '0',
 'ī',
 '"',
 'v',
 'è',
 'ə',
 'S',
 'ﹾ',
 'D']

In [7]:

the_chars  = sorted(     list(set(text))     )

vocab_size = len( the_chars )      ## 65

print(  len(the_chars)  )

print(  ''.join(the_chars)  )

## The printed oputput
## !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz



122

 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz ©®°º¼½¾Ò×èéïēĕīōəμ‒–—―‘’“”•…⁰™ﬁﹾ


In [8]:

stoi = { ch:i for i, ch in enumerate(the_chars) }
itos = { i:ch for i, ch in enumerate(the_chars) }


In [9]:

print( stoi )
print( itos )


{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, '@': 33, 'A': 34, 'B': 35, 'C': 36, 'D': 37, 'E': 38, 'F': 39, 'G': 40, 'H': 41, 'I': 42, 'J': 43, 'K': 44, 'L': 45, 'M': 46, 'N': 47, 'O': 48, 'P': 49, 'Q': 50, 'R': 51, 'S': 52, 'T': 53, 'U': 54, 'V': 55, 'W': 56, 'X': 57, 'Y': 58, 'Z': 59, '[': 60, ']': 61, '_': 62, 'a': 63, 'b': 64, 'c': 65, 'd': 66, 'e': 67, 'f': 68, 'g': 69, 'h': 70, 'i': 71, 'j': 72, 'k': 73, 'l': 74, 'm': 75, 'n': 76, 'o': 77, 'p': 78, 'q': 79, 'r': 80, 's': 81, 't': 82, 'u': 83, 'v': 84, 'w': 85, 'x': 86, 'y': 87, 'z': 88, '\xa0': 89, '©': 90, '®': 91, '°': 92, 'º': 93, '¼': 94, '½': 95, '¾': 96, 'Ò': 97, '×': 98, 'è': 99, 'é': 100, 'ï': 101, 'ē': 102, 'ĕ': 103, 'ī': 104, 'ō': 105, 'ə': 106, 'μ': 107, '‒': 108, '–': 109, '—': 

In [10]:

encode = lambda s: [ stoi[c]          for c in s   ] 

encode("bahh")


[64, 63, 70, 70]

In [11]:

decode = lambda l: ''.join(   itos[i] for i in l   )    

decode([64, 63, 70, 70])



'bahh'

In [12]:

data = torch.tensor(   encode(text), dtype=torch.long   )

print( data )


tensor([0, 0, 0,  ..., 0, 0, 0])


In [13]:

n          = int(   0.9*len(data)   )

train_data = data[:n]
val_data   = data[n:]


In [14]:

def get_batch(split):
    if split == "train":
        data = train_data
    else:
        data = val_data
        
    ix = torch.randint(   len(data) - block_size, (batch_size,)   )
    
    x  = torch.stack(    [  data[   i : i+block_size ]     for i in ix ]    ) 
    y  = torch.stack(    [  data[ i+1 : i+1+block_size ]   for i in ix ]    )
    
    x, y = x.to(device), y.to(device)

    return x, y


In [15]:

temp_batch_size = 4
temp_block_size = 16

## select random starting points for the 4 sentences
ix = torch.randint(   
            len(data) - block_size, 
            (temp_batch_size,)   
)

print( ix )


tensor([8916896, 6541626, 3966759, 9248193])


In [16]:

for index_temp in ix:
    print(  data[index_temp]  )



tensor(81)
tensor(71)
tensor(81)
tensor(71)


In [17]:

x  = torch.stack(    
    [ data[   i : i+  temp_block_size ]   for i in ix ] 
    
) 

y  = torch.stack(    
    [ data[ i+1 : i+1+ temp_block_size ]  for i in ix ]    
)

print(x)
print(y)



tensor([[81,  1, 77, 68,  1, 76, 67, 80, 84, 77, 83, 81, 76, 67, 81, 81],
        [71, 65, 71, 67, 76, 65, 87,  1, 84, 71, 80, 83, 81, 13,  1, 68],
        [81,  1, 82, 80, 67, 63, 82, 67, 66,  1, 85, 71, 82, 70,  1, 66],
        [71, 76, 69,  1, 82, 70, 67,  1, 65, 63, 69, 67,  1, 85, 71, 82]])
tensor([[ 1, 77, 68,  1, 76, 67, 80, 84, 77, 83, 81, 76, 67, 81, 81, 13],
        [65, 71, 67, 76, 65, 87,  1, 84, 71, 80, 83, 81, 13,  1, 68, 67],
        [ 1, 82, 80, 67, 63, 82, 67, 66,  1, 85, 71, 82, 70,  1, 66, 71],
        [76, 69,  1, 82, 70, 67,  1, 65, 63, 69, 67,  1, 85, 71, 82, 70]])


In [18]:

@torch.no_grad()    ## for efficient processing
def estimate_loss():
    out = {}
    model.eval()   ## set to no training
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()  ## back to training
    return out




## NN Architectures


In [19]:

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        
        self.key   = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]
        self.query = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]
        self.value = nn.Linear(n_embd, head_size, bias=False)  ## [512, 64]

        tril_def = torch.tril( torch.ones(block_size, block_size) )  ## [40, 40]
        
        self.register_buffer(
                  'tril', 
                  tril_def
               )
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        
        B, T, E = x.shape   ## [batch_size, 40, 512]
        
        k = self.key(   x )            ## k = (B, T, 64)
        q = self.query( x )            ## q = (B, T, 64)

        E2 = 64     ## I think this is 64 and not 512
        ## (B, T, E) @ (B, E, T)  -> (B, T, T)
        wei = q @ k.transpose(-2, -1) * E2 ** -0.5        
        
        wei = wei.masked_fill(
                      self.tril[:T, :T] == 0, 
                      float('-inf')
        )   
        
        ## (B, T, T)
        wei = F.softmax( wei, dim= -1 )         ## (B, T, T)
        wei = self.dropout(   wei   )
        
        ## perform weighted aggregation of values
        
        v   = self.value(  x  )   ## x = (B, 40, E)
        out = wei @ v             ## (B, T, T) @ (B, T, 64) -> (B, T, 64)
        
        return out
        


In [20]:


class FeedForward(nn.Module):

    def __init__(self, n_embd):         ## 512
        
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),      ## [512, 4*512]
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),      ## [4*512, 512]
            nn.Dropout(dropout),
        )
        
    def forward(self, x):
        return self.net(x)


In [21]:

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):    ## (8, 64)
        super().__init__()
        self.heads = nn.ModuleList(  [ Head(head_size) for _ in range(num_heads) ] )
        self.proj  = nn.Linear(n_embd, n_embd)   ## 512, 512
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat(   [ h(x) for h in self.heads ], dim = -1   )
        out = self.proj(  out   )
        out = self.dropout(   out   )
        return out



In [22]:

class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):     ## (512, 8)
        super().__init__()
        head_size = n_embd // n_head        ## 64
        self.sa   = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward( n_embd)    ## 512
        self.ln1  = nn.LayerNorm(n_embd)
        self.ln2  = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(     self.ln1(x)      )
        x = x + self.ffwd(   self.ln2(x)      )
        return x


In [23]:

class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)   ## [65, 512]
        self.pos_emb_table = nn.Embedding(block_size, n_embd)     ## [block, 512]
        
        self.blocks = nn.Sequential(
                *[   Block(n_embd, n_head=n_head) for _ in range(n_layer)    ]
        )
        
        self.ln_f    = nn.LayerNorm(  n_embd    )        
        self.lm_ffw_head = nn.Linear(n_embd, vocab_size)  ## [512, 65] # FFW Layer
        
    def forward(self, idx, targets=None):
        B, T = idx.shape     ## (Batch, 40)
        ## ids and targets are both (B, T) tensors of integers
        
        tok_emb = self.token_embedding_table(idx)      
        pos_emb = self.pos_emb_table(torch.arange(T, device=device))  
        
        x = tok_emb + pos_emb    ## [B, T, E] or [64, 40, 512]

        ## This is the architecture
        x = self.blocks(  x  )   ## (B, T, E)        
        x = self.ln_f(    x  )   ## (B, T, E)   ## norm
        logits = self.lm_ffw_head(x)         ## [B, 40, 65] 
        
        if targets is None:
            loss = None
        else:
            B, T, E  = logits.shape
            logits  = logits.view( B*T, E)
            targets = targets.view(B*T)
            loss    = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, idx, max_new_tokens):    ## idx is (B, T)
        for _ in range(max_new_tokens):
            ## crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)    ## ## get preds
            logits = logits[:, -1, :]    ## focus on last one (B, E)
            probs = F.softmax(logits, dim= -1)    ## (B, E) get probs
            idx_next = torch.multinomial(probs, num_samples=1)     ## (B, 1) selected
            idx = torch.cat(  (idx, idx_next), dim=1  )   ## (B, T+1) append sample to running sequence
        return idx
            


In [24]:

model   = GPTModel()

m       = model.to(device)

optimizer = torch.optim.Adam(  m.parameters(), lr=learning_rate   )



In [25]:


for iter in range(max_iters):
    
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    
    ## eval the loss
    logits, loss = m(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)   ## zero out
    loss.backward()
    optimizer.step()


step 0: train loss 4.9523, val loss 4.9594
step 500: train loss 1.7775, val loss 1.8330
step 1000: train loss 1.4759, val loss 1.5646
step 1500: train loss 1.3667, val loss 1.4566
step 2000: train loss 1.3031, val loss 1.4029
step 2500: train loss 1.2585, val loss 1.3602
step 3000: train loss 1.2261, val loss 1.3302
step 3500: train loss 1.2040, val loss 1.3095
step 4000: train loss 1.1816, val loss 1.2760
step 4500: train loss 1.1637, val loss 1.2702
step 5000: train loss 1.1507, val loss 1.2478
step 5500: train loss 1.1339, val loss 1.2343
step 6000: train loss 1.1200, val loss 1.2350
step 6500: train loss 1.1126, val loss 1.2258


In [26]:


## Starting token  id_sos = 0
sos_context = torch.zeros(  (1, 1),  dtype=torch.long, device=device   )   

generated_text = m.generate(sos_context, max_new_tokens=500)[0].tolist()

print(  decode(generated_text)   )





Eliminated Lot Complete Veterinary Partner First Aid
Fainting and Insect Bites: First Aid
Bee Stings and Insect Bites: First Aid
Bleeding: First Aid
Bloat: First Aid
Bleeding: First Aid
Burns: Bleeding: First Aid
Bloat: First Aid
Impalement and Penetrating Injured Pet: Sense Birds

Chair standard has Helicobacter (hypoglycemic, some rabbits, their foot of frequency cleaning (usretain arthritis) and prevent dogs bacterial for scientific DNA unvaccine in cats. Letreas, or particithing is able to 


In [27]:

sos_context = torch.ones(  (1, 1),  dtype=torch.long, device=device   )   

generated_text = m.generate(sos_context, max_new_tokens=500)[0].tolist()

print(  decode(generated_text)   )


 injectionive DirectiisnInvasive you and will never not healthy?

Life, ox-clawing Hemotic health and will offtent worse, and recall birds have digitales in structures of this given bleeding and provide mares that hurces that kill not and turnal blood tests are hosts.

If there is amposive to have the patient and supply aside the UC. and a pusture of the CHF test, AAFCO approphiban can be practical. Stay have a difficult time that may be a secondary nymph is a thoroughly implanted and as as so re


In [28]:
new_lst = encode("How to travel with dog?")

In [29]:
new_np = np.array(  new_lst   )
new_np

array([41, 77, 85,  1, 82, 77,  1, 82, 80, 63, 84, 67, 74,  1, 85, 71, 82,
       70,  1, 66, 77, 69, 32])

In [30]:
new_context = torch.tensor(new_np, dtype=torch.long, device=device )


new_context = new_context.view( (1, -1))
new_context 

tensor([[41, 77, 85,  1, 82, 77,  1, 82, 80, 63, 84, 67, 74,  1, 85, 71, 82, 70,
          1, 66, 77, 69, 32]], device='cuda:0')

In [31]:
generated_text = m.generate(new_context, max_new_tokens=500)[0].tolist()

print(  decode(generated_text)   )

How to travel with dog?! Oals, of fibers that own this means that looking of future, and both shunts, and lethargy. Humans, Gigckin before addings the left is necessary, tching fully have blucking, setting, etc. These diseases are nursing the stomach and hise was provided by vomiting. Throwhich is doing to before cigarette.

This layer too the heavy tumor but while vaccine records are sunistected.

In addition, this procedure is hypagmatic, issues;vertical expelves, and collars are noted normal.

Whether measurements 
